# Setup

In [2]:
import os
import sys
import time
import argparse
import traceback
import getpass
import json
from typing import List, Optional, Tuple
import concurrent
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.parse
import io
from tqdm.auto import tqdm

In [3]:
class CoralNetDownloader:
    """Main downloader class for CoralNet sources using requests"""
    
    CORALNET_URL = "https://coralnet.ucsd.edu"
    LOGIN_URL = "https://coralnet.ucsd.edu/accounts/login/"
    
    def __init__(self, username: str, password: str):
        self.username = username
        self.password = password
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        })
        self.logged_in = False
    
    def login(self) -> bool:
        """Log in to CoralNet using requests session"""
        success = False
        try:
            # Get login page to extract CSRF token
            response = self.session.get(self.LOGIN_URL, timeout=30)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            csrf_token = soup.find("input", attrs={"name": "csrfmiddlewaretoken"})
            
            if not csrf_token:
                raise Exception("Could not find CSRF token")
            
            # Prepare login data
            data = {
                "username": self.username,
                "password": self.password,
                "csrfmiddlewaretoken": csrf_token["value"],
            }
            
            headers = {"Referer": self.LOGIN_URL}
            
            # Submit login
            login_response = self.session.post(
                self.LOGIN_URL, 
                data=data, 
                headers=headers,
                timeout=30,
                allow_redirects=True
            )
            
            # Check if login was successful by looking for sign out button or redirect
            if "Sign out" in login_response.text or login_response.url != self.LOGIN_URL:
                success = True
                self.logged_in = True
                print("✓ Login successful")
            else:
                raise Exception("Login failed - invalid credentials or other error")
                
        except Exception as e:
            print(f"ERROR: Could not login with {self.username}: {str(e)}")
        
        return success
    
    def check_permissions(self, source_id: int) -> bool:
        """Check permissions for accessing a source"""
        try:
            url = f"{self.CORALNET_URL}/source/{source_id}/"
            response = self.session.get(url, timeout=30)
            response.raise_for_status()
            
            if "Page could not be found" in response.text:
                raise Exception("Source does not exist")
            elif "don't have permission" in response.text:
                raise Exception("Permission denied")
            
            return True
            
        except Exception as e:
            print(f"ERROR: Permission check failed for source {source_id}: {str(e)}")
            return False
    
    def download_metadata(self, source_id: int, output_dir: str) -> Tuple[bool, int]:
        """Download metadata for a source"""
        success = False
        total_images_number = 0
        
        try:
            url = f"{self.CORALNET_URL}/source/{source_id}/"
            response = self.session.get(url, timeout=30)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Try to get total images count
            try:
                image_status_header = soup.find("h4", string="Image Status")
                if image_status_header:
                    table = image_status_header.find_next_sibling("table", class_="detail_box_table")
                    if table:
                        # Find the row where one of the <td> contains 'Total images:'
                        total_images_row = None
                        for tr in table.find_all("tr"):
                            tds = tr.find_all("td")
                            if any("Total images:" in td.get_text() for td in tds):
                                total_images_row = tr
                                break
                        if total_images_row:
                            link = total_images_row.find("a")
                            if link:
                                try:
                                    total_images_number = int(link.get_text().strip().replace(",", ""))
                                except Exception:
                                    total_images_number = 0
                                print(f"Total images: {total_images_number}")
            except Exception as e:
                print(f"Warning: Can't get number of images: {e}")
                total_images_number = 0
            
            # Extract classifier plot data from JavaScript
            script_tags = soup.find_all("script")
            classifier_data = None
            
            for script in script_tags:
                if script.string and "Classifier overview" in script.string:
                    script_text = script.string
                    start_marker = "let classifierPlotData = "
                    start_index = script_text.find(start_marker)
                    
                    if start_index != -1:
                        start_index += len(start_marker)
                        end_index = script_text.find("];", start_index) + 1
                        classifier_plot_data_str = script_text[start_index:end_index]
                        
                        # Clean up JavaScript object notation to valid JSON
                        classifier_plot_data_str = classifier_plot_data_str.replace("'", '"')
                        
                        try:
                            classifier_data = json.loads(classifier_plot_data_str)
                            break
                        except json.JSONDecodeError as e:
                            print(f"Warning: Could not parse classifier data: {e}")
            
            if not classifier_data:
                print("No metadata found for this source")
                return True, total_images_number
            
            # Process classifier data
            meta = []
            for point in classifier_data:
                meta.append([
                    point.get("x"),        # classifier_nbr
                    point.get("y"),        # score
                    point.get("nimages"),  # nimages
                    point.get("traintime"), # traintime
                    point.get("date"),     # date
                    point.get("pk")        # src_id
                ])
            
            # Save metadata
            meta_df = pd.DataFrame(meta, columns=[
                'Classifier nbr', 'Accuracy', 'Trained on',
                'Date', 'Traintime', 'Global id'
            ])
            print(meta_df)
            filepath = os.path.join(output_dir, "metadata.csv")
            meta_df.to_csv(filepath, index=False)
            
            if os.path.exists(filepath):
                print(f"✓ Metadata saved to {filepath}")
                success = True
            else:
                raise Exception("Metadata could not be saved")
                
        except Exception as e:
            print(f"ERROR: Issue downloading metadata: {str(e)}")
        
        return success, total_images_number
    
    def download_labelset(self, source_id: int, output_dir: str) -> bool:
        """Download labelset for a source"""
        success = False
        
        try:
            url = f"{self.CORALNET_URL}/source/{source_id}/labelset/"
            response = self.session.get(url, timeout=30)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', {'id': 'label-table'})
            
            if table is None:
                raise Exception("Unable to find the label table")
            
            rows = table.find_all('tr')
            if not rows or len(rows) <= 1:  # Only header row or no rows
                print("No labelset found for this source")
                return True
            
            label_ids = []
            names = []
            short_codes = []
            
            for row in rows[1:]:  # Skip header row
                cells = row.find_all('td')
                if cells:
                    # Get label ID from link
                    link = cells[0].find('a')
                    if link and link.get('href'):
                        label_id = link['href'].split('/')[-2]
                        label_ids.append(label_id)
                        
                        # Get name
                        names.append(link.get_text().strip())
                        
                        # Get short code (second column)
                        if len(cells) > 1:
                            short_codes.append(cells[1].get_text().strip())
                        else:
                            short_codes.append("")
            
            if label_ids:
                labelset_df = pd.DataFrame({
                    'Label ID': label_ids,
                    'Name': names,
                    'Short Code': short_codes
                })
                
                filepath = os.path.join(output_dir, "labelset.csv")
                labelset_df.to_csv(filepath, index=False)
                
                if os.path.exists(filepath):
                    print(f"✓ Labelset saved to {filepath}")
                    success = True
                else:
                    raise Exception("Labelset could not be saved")
            else:
                print("No labels found in labelset")
                success = True
                
        except Exception as e:
            print(f"ERROR: Issue downloading labelset: {str(e)}")
        
        return success
    
    def download_annotations(self, source_id: int, output_dir: str, n_images: int) -> bool:
        """Download annotations for a source"""
        success = False
        
        try:
            # First, get the browse images page to extract form data
            browse_url = f"{self.CORALNET_URL}/source/{source_id}/browse/images/"
            response = self.session.get(browse_url, timeout=30)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find the export form
            export_form = soup.find('form', {'id': 'export-annotations-prep-form'})
            if not export_form:
                raise Exception("Could not find export annotations form")
            
            # Extract CSRF token
            csrf_token = export_form.find('input', {'name': 'csrfmiddlewaretoken'})
            if not csrf_token:
                raise Exception("Could not find CSRF token in export form")
            
            # Prepare form data for annotation export
            form_data = {
                'csrfmiddlewaretoken': csrf_token['value'],
                'browse_action': 'export_annotations',
                'image_select_type': 'all',
                'label_format': 'both',
                # Add all optional columns
                'optional_columns': [
                    'annotator_info',
                    "metadata_date_aux",
                    "metadata_other",
                ]
            }
            
            export_request_url = f"{self.CORALNET_URL}/source/{source_id}/annotation/export_prep/"
            # Submit the export request
            export_response = self.session.post(
                export_request_url,
                headers={'Referer': browse_url},
                data=form_data,
                timeout=120,  # Longer timeout for processing
                allow_redirects=True
            )
            
            export_timestamp = export_response.json()['session_data_timestamp']
            download_annotations_url = f"https://coralnet.ucsd.edu/source/{source_id}/export/serve/?session_data_timestamp={export_timestamp}"
            download_annotations_response = self.session.get(download_annotations_url, timeout=60)
            download_annotations_response.raise_for_status()

            df_annotations = pd.read_csv(io.StringIO(download_annotations_response.text))
            annotations_file = os.path.join(output_dir, "annotations.csv")
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            df_annotations.to_csv(annotations_file, index=False)
            if os.path.exists(annotations_file) and os.path.getsize(annotations_file) > 0:
                print(f"✓ Annotations saved to {annotations_file}")
                success = True
            else:
                raise Exception("Downloaded annotations file is empty")

        except Exception as e:
            print(f"ERROR: Issue downloading annotations: {str(e)}")
            success = False  # Don't fail the entire process
        
        return success
    
    def get_images_on_page(self, browse_url) -> tuple[dict[str, str], Optional[str]]:
        """
        Get a dictionary of image names and their URLs from the CoralNet browse page
        
        Args:
            session: requests.Session object with valid CoralNet login
            browse_url: URL of the browse images page
            
        Returns:
            dict: Dictionary with image names as keys and their URLs as values
        """
        images = {}
        
        response = self.session.get(browse_url, timeout=30)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        
        thumb_wrappers = soup.find_all('span', class_='thumb_wrapper')
        for wrapper in thumb_wrappers:
            link = wrapper.find('a')
            img = wrapper.find('img')
            if link and img:
                image_name = img.get('title', '')
                image_url = link.get('href', '')
                
                if image_name and image_url:
                    images[image_name] = image_url

        next_page_element = soup.find('a', title='Next page')
        next_page_url = next_page_element.get('href') if next_page_element else None
                    
        return images, next_page_url

    def get_images(self, source_id) -> Tuple[Optional[pd.DataFrame], bool]:
        """
        Get a DataFrame of all images from a CoralNet source
        
        Args:
            session: requests.Session object with valid CoralNet login
            source_id: ID of the CoralNet source

        Returns:
            pd.DataFrame: DataFrame containing image names and URLs
        """
        images = None
        success = False

        base_url = f'{self.CORALNET_URL}/source/{source_id}/browse/images'
        all_images = {}
        try: 
            imgs, next_page = self.get_images_on_page(base_url)
            all_images.update(imgs)
            p_bar = tqdm(desc="Fetching images", unit="page")
            while next_page:
                imgs, next_page = self.get_images_on_page(f"{base_url}/{next_page}")
                all_images.update(imgs)
                p_bar.update(1)
            p_bar.close()
            success = True
            images = pd.DataFrame(list(all_images.items()), columns=['Name', 'Image Page'])
        except Exception as e:
            print(f"ERROR: Issue retrieving images: {str(e)}")
        return images, success

    # def get_images(self, source_id: int) -> Tuple[Optional[pd.DataFrame], bool]:
        """Get list of images from a source"""
        images = None
        success = False
        
        try:
            base_url = f"{self.CORALNET_URL}/source/{source_id}/browse/images/"
            
            # Get first page to determine total pages
            response = self.session.get(base_url, timeout=30)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find total images count
            line_div = soup.find('div', class_='line')
            if line_div:
                line_text = line_div.get_text()
                # Extract total from text like "Showing 1 to 20 of 150 images"
                parts = line_text.split()
                if 'of' in parts:
                    total_images = int(parts[parts.index('of') + 1])
                    total_pages = (total_images + 19) // 20  # Round up, 20 images per page
                    print(f"Found {total_images} images across {total_pages} pages")
                else:
                    total_images = 20  # Assume at least one page
                    total_pages = 1
            else:
                total_images = 20
                total_pages = 1
            
            image_page_urls = []
            image_names = []
            
            # Collect images from all pages
            for page in range(1, total_pages + 1):
                time.sleep(1)  # Rate limiting
                
                if page > 1:
                    page_url = f"{base_url}?page={page}"
                else:
                    page_url = base_url
                
                try:
                    page_response = self.session.get(page_url, timeout=30)
                    page_response.raise_for_status()
                    
                    page_soup = BeautifulSoup(page_response.text, 'html.parser')
                    
                    # Find thumbnail links and images
                    thumb_wrappers = page_soup.find_all('div', class_='thumb_wrapper')
                    
                    for wrapper in thumb_wrappers:
                        link = wrapper.find('a')
                        img = wrapper.find('img')
                        
                        if link and img:
                            image_page_urls.append(urllib.parse.urljoin(self.CORALNET_URL, link['href']))
                            image_names.append(img.get('alt', ''))
                
                except Exception as e:
                    print(f"Warning: Failed to get page {page}: {e}")
                    time.sleep(5)  # Wait longer on error
                    continue
            
            if image_names and image_page_urls:
                images = pd.DataFrame({
                    'Name': image_names,
                    'Image Page': image_page_urls
                })
                print(f"✓ Found {len(images)} images")
                success = True
            else:
                print("No images found")
                success = True
                
        except Exception as e:
            print(f"ERROR: Issue retrieving images: {str(e)}")
        
        return images, success

    def get_image_urls(self, image_page_urls: List[str]) -> List[Optional[str]]:
        """
        Get the direct image URLs from the CoralNet image page URLs

        Args:
            image_page_url: URL of the CoralNet image page

        Returns:
            str or None: Direct image URL or None if not found
        """
        image_urls = []
        for image_page_url in tqdm(image_page_urls, desc="Fetching image URLs", unit="image"):
            image_page_url = f'https://coralnet.ucsd.edu{image_page_url}'
            image_view_response = urllib.request.urlopen(image_page_url)
            response_soup = BeautifulSoup(
                image_view_response.read(), 'html.parser')

            original_img_elements = response_soup.select(
                'div#original_image_container > img')
            if not original_img_elements:
                raise ValueError(
                    f"CoralNet image {image_page_url}: couldn't find image on the"
                    f" image-view page. Maybe it's in a private source.")
            image_url = original_img_elements[0].attrs.get('src')
            image_urls.append(image_url)

        return image_urls
    
    @staticmethod
    def download_image(url: str, path: str, timeout: int = 30) -> Tuple[str, bool]:
        """Download a single image"""
        if os.path.exists(path):
            return path, True
        
        try:
            response = requests.get(url, timeout=timeout, stream=True)
            response.raise_for_status()
            
            os.makedirs(os.path.dirname(path), exist_ok=True)
            
            with open(path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            
            if os.path.exists(path) and os.path.getsize(path) > 0:
                return path, True
            else:
                return path, False
                
        except Exception as e:
            print(f"Warning: Failed to download {url}: {e}")
            return path, False
    
    def download_images(self, images_df: pd.DataFrame, output_dir: str, source_id: int):
        """Download all images from a DataFrame"""
        # Save image list
        csv_file = os.path.join(output_dir, "images.csv")
        images_df.to_csv(csv_file, index=False)
        print(f"✓ Saved image list to {csv_file}")
        
        # Create images directory
        image_dir = os.path.join(output_dir, "images")
        os.makedirs(image_dir, exist_ok=True)
        
        # Filter out rows without URLs
        valid_images = images_df[images_df['Image URL'].notna()]
        
        if valid_images.empty:
            print("Warning: No valid image URLs found")
            return
        
        print(f"Downloading {len(valid_images)} images...")
        
        with ThreadPoolExecutor(max_workers=min(8, os.cpu_count() or 4)) as executor:
            futures = []
            for _, row in valid_images.iterrows():
                name = row['Name']
                url = row['Image URL']
                path = os.path.join(image_dir, name)
                futures.append(executor.submit(self.download_image, url, path))
            
            completed = 0
            successful = 0
            for future in concurrent.futures.as_completed(futures):
                try:
                    path, success = future.result()
                    if success:
                        successful += 1
                    else:
                        print(f"Warning: Failed to download {os.path.basename(path)}")
                except Exception as e:
                    print(f"ERROR: {str(e)}")
                
                completed += 1
                if completed % 10 == 0 or completed == len(futures):
                    print(f"Progress: {completed}/{len(futures)} images processed")
        
        print(f"✓ Downloaded {successful}/{len(valid_images)} images to {image_dir}")
    
    def download_source(self, source_id: int, output_dir: str,
                       download_metadata: bool = True,
                       download_labelset: bool = True,
                       download_annotations: bool = True,
                       download_images: bool = True) -> bool:
        """Download all data for a source"""
        print(f"\n=== Downloading Source {source_id} ===")
        
        # Create source directory
        source_dir = os.path.join(output_dir, str(source_id))
        os.makedirs(source_dir, exist_ok=True)
        
        # Login if needed
        if not self.logged_in:
            if not self.login():
                raise Exception("Failed to login to CoralNet")
        
        # Check permissions
        if not self.check_permissions(source_id):
            raise Exception(f"Cannot access source {source_id}")
        
        success = True
        n_images = 0
        
        # Download metadata
        if download_metadata:
            metadata_success, n_images = self.download_metadata(source_id, source_dir)
            if not metadata_success:
                print("Warning: Failed to download metadata")
            
            if n_images == 0:
                print("Source appears to be empty, creating empty marker")
                os.makedirs(os.path.join(source_dir, "empty"), exist_ok=True)
                return True
        
        # Download labelset
        if download_labelset:
            if not self.download_labelset(source_id, source_dir):
                print("Warning: Failed to download labelset")
        
        # Download annotations
        if download_annotations:
            if not self.download_annotations(source_id, source_dir, n_images):
                print("Warning: Failed to download annotations")
        
        # Download images
        if download_images:
            images_df, images_success = self.get_images(source_id)
            if images_success and images_df is not None and len(images_df) > 0:
                # Get image URLs
                image_urls = self.get_image_urls(images_df['Image Page'].tolist())
                images_df['Image URL'] = image_urls
                
                # Download images
                self.download_images(images_df, source_dir, source_id)
            else:
                print("Warning: No images found or failed to retrieve image list")
        
        print(f"✓ Completed downloading source {source_id}")
        return success
    
    def cleanup(self):
        """Clean up resources"""
        if self.session:
            self.session.close()
        self.logged_in = False


In [ ]:
# def main():
#     parser = argparse.ArgumentParser(
#         description="Download data from CoralNet sources using requests",
#         formatter_class=argparse.RawDescriptionHelpFormatter,
#         epilog="""
# Examples:
#   %(prog)s 123 -o ./downloads
#   %(prog)s 123,456,789 -o ./data --no-images
#   %(prog)s 123 -o ./output --metadata-only
#   %(prog)s 123 -u myuser -p mypass -o ./downloads
# """
#     )
    
#     parser.add_argument('source_ids',
#                        help='Comma-separated list of source IDs to download')
#     parser.add_argument('-o', '--output', required=True,
#                        help='Output directory for downloads')
#     parser.add_argument('-u', '--username',
#                        help='CoralNet username (if not provided, will prompt)')
#     parser.add_argument('-p', '--password',
#                        help='CoralNet password (if not provided, will prompt)')
    
#     # Download options
#     parser.add_argument('--no-metadata', action='store_true',
#                        help='Skip metadata download')
#     parser.add_argument('--no-labelset', action='store_true',
#                        help='Skip labelset download')
#     parser.add_argument('--no-annotations', action='store_true',
#                        help='Skip annotations download')
#     parser.add_argument('--no-images', action='store_true',
#                        help='Skip images download')
    
#     # Convenience flags
#     parser.add_argument('--metadata-only', action='store_true',
#                        help='Download only metadata')
#     parser.add_argument('--annotations-only', action='store_true',
#                        help='Download only annotations')
    
#     args = parser.parse_args()
    
#     # Parse source IDs
#     try:
#         source_ids = [int(s.strip()) for s in args.source_ids.split(',')]
#     except ValueError:
#         print("ERROR: Source IDs must be comma-separated integers")
#         sys.exit(1)
    
#     # Get credentials
#     username = "ViktorDo" #args.username or input("CoralNet username: ")
#     password = "ThACU74QW7iEV@F" #args.password or getpass.getpass("CoralNet password: ")
    
#     if not username or not password:
#         print("ERROR: Username and password are required")
#         sys.exit(1)
    
#     # Set download options
#     if args.metadata_only:
#         download_metadata = True
#         download_labelset = False
#         download_annotations = False
#         download_images = False
#     elif args.annotations_only:
#         download_metadata = False
#         download_labelset = False
#         download_annotations = True
#         download_images = False
#     else:
#         download_metadata = not args.no_metadata
#         download_labelset = not args.no_labelset
#         download_annotations = not args.no_annotations
#         download_images = not args.no_images
    
#     # Create output directory
#     output_dir = os.path.abspath(args.output)
#     os.makedirs(output_dir, exist_ok=True)
    
#     print(f"CoralNet CLI Downloader (Requests Version)")
#     print(f"Output directory: {output_dir}")
#     print(f"Source IDs: {source_ids}")
#     print(f"Options: metadata={download_metadata}, labelset={download_labelset}, "
#           f"annotations={download_annotations}, images={download_images}")
    
#     # Initialize downloader
#     downloader = CoralNetDownloader(username=username, password=password)
    
#     try:
#         # Download each source
#         for source_id in source_ids:
#             try:
#                 downloader.download_source(
#                     source_id=source_id,
#                     output_dir=output_dir,
#                     download_metadata=download_metadata,
#                     download_labelset=download_labelset,
#                     download_annotations=download_annotations,
#                     download_images=download_images
#                 )
#             except Exception as e:
#                 print(f"ERROR: Failed to download source {source_id}: {str(e)}")
#                 traceback.print_exc()
#                 continue
        
#         print("\n=== Download Complete ===")
        
#     except KeyboardInterrupt:
#         print("\nDownload interrupted by user")
#     except Exception as e:
#         print(f"ERROR: {str(e)}")
#         traceback.print_exc()
#         sys.exit(1)
#     finally:
#         downloader.cleanup()

# Run Model Locally

In [8]:
# Get credentials
username = input("CoralNet username: ")
password = getpass.getpass("CoralNet password: ")

In [6]:
source_id = "5027"
output_dir = "./scrape-res"
source_dir = os.path.join(output_dir, str(source_id))
os.makedirs(source_dir, exist_ok=True)

In [ ]:
downloader = CoralNetDownloader(username=username, password=password)
downloader.login()

✓ Login successful


True

In [11]:
metadata_success, n_images = downloader.download_metadata(source_id, source_dir)
downloader.download_labelset(source_id, source_dir)
downloader.download_annotations(source_id, source_dir, n_images)

Total images: 40
   Classifier nbr  Accuracy  Trained on     Date     Traintime Global id
0               1        69          22  0:00:02  May 18, 2024     44953
1               2        80          35  0:00:03  May 18, 2024     44955
✓ Metadata saved to ./scrape-res/5027/metadata.csv
✓ Labelset saved to ./scrape-res/5027/labelset.csv
✓ Annotations saved to ./scrape-res/5027/annotations.csv


True

In [12]:
images_df, images_success = downloader.get_images(source_id)
if images_success and images_df is not None and len(images_df) > 0:
    # Get image URLs
    image_urls = downloader.get_image_urls(images_df['Image Page'].tolist())
    images_df['Image URL'] = image_urls

    downloader.download_images(images_df, source_dir, source_id)

Fetching images: 0page [00:00, ?page/s]

Fetching image URLs:   0%|          | 0/40 [00:00<?, ?image/s]

✓ Saved image list to ./scrape-res/5027/images.csv
Progress: 10/40 images processed
Progress: 20/40 images processed
Progress: 30/40 images processed
Progress: 40/40 images processed
✓ Downloaded 40/40 images to ./scrape-res/5027/images
